# Headers

In [1]:
import pandas as pd


import os
import dotenv
# Specify the path to your .env file
env_file_path = '/home/azureuser/cloudfiles/code/Users/vlavanga/.env' 

# Reload the variables and override existing ones
dotenv.load_dotenv(env_file_path, override=True)



True

# Connec & Query Snowflake

## Make Connection

In [2]:
password = os.getenv("SNOWFLAKE_PASSWORD")
user=os.getenv("SNOWFLAKE_USER")
account=os.getenv("SNOWFLAKE_ACCOUNT")
warehouse=os.getenv("SNOWFLAKE_WAREHOUSE")
database=os.getenv("SNOWFLAKE_DATABASE")
schema='PUBLIC'

## Query Data

In [5]:
import snowflake.connector

In [6]:

conn = snowflake.connector.connect(
    user=user,
    password=password,
    account=account,
    warehouse=warehouse,
    database=database,
    schema=schema
)

In [7]:
snowflake_query_base = pd.read_sql(
    '''
WITH deduped AS (SELECT DISTINCT * 
FROM NECCDMDEV.FACTS.FACT_PURCHASE_ORDER_ANALYSIS)
SELECT 
    po.PO_MATERIAL_KEY,
    po.PURCHASE_ORDER_DOCUMENT,
    po.PO_CREATED_DATE,
    po.SALES_ORG,
    dm.FABRIC_CONTENT_CODE_TEXT,
    dm.TEAM,
    dm.SILHOUETTE,
    dm.SPORT,
    dm.DIVISION_NAME,
    dm.EAN_UPC,
    dm.MARKET_SEGMENT,
    dm.GENDER,
    ds.SEASON_CONSOLIDATION,
    po.ORDERED_QUANTITY
FROM 
    deduped AS po
JOIN 
    NECCDMDEV.DIMENSIONS.DIM_MATERIAL AS dm
    ON po.PO_MATERIAL_KEY = dm.MATERIAL_KEY
JOIN 
    NECCDMDEV.DIMENSIONS.DIM_SEASON_COLLECTION AS ds
    ON po.SEASON_COLLECTION_KEY = ds.SEASON_COLLECTION_KEY;
''', 
conn)

/tmp/ipykernel_5552/1052042120.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  snowflake_query_base = pd.read_sql(


In [ ]:
snowflake_query_v4 = pd.read_sql(
    '''
    SELECT *
FROM 
    NECCDMDEV.FACTS.FACT_PURCHASE_ORDER_ANALYSIS AS po
JOIN 
    NECCDMDEV.DIMENSIONS.DIM_MATERIAL AS dm
    ON po.PO_MATERIAL_KEY = dm.MATERIAL_KEY
JOIN 
    NECCDMDEV.DIMENSIONS.DIM_SEASON_COLLECTION AS ds
    ON po.SEASON_COLLECTION_KEY = ds.SEASON_COLLECTION_KEY;
''', 
conn)

In [79]:
snowflake_query_v4 = pd.read_sql(
    '''
    WITH deduped_po AS (
        SELECT DISTINCT *
        FROM NECCDMDEV.FACTS.FACT_PURCHASE_ORDER_ANALYSIS
    )
    SELECT 
    po.PO_MATERIAL_KEY,
    po.PURCHASE_ORDER_DOCUMENT,
    po.PO_CREATED_DATE,
    po.SALES_ORG,
    dm.FABRIC_CONTENT_CODE_TEXT,
    dm.TEAM,
    dm.SILHOUETTE,
    dm.SPORT,
    dm.DIVISION_NAME,
    dm.EAN_UPC,
    ds.SEASON_CONSOLIDATION,
    po.ORDERED_QUANTITY
FROM 
    deduped_po AS po
JOIN 
    NECCDMDEV.DIMENSIONS.DIM_MATERIAL AS dm
    ON po.PO_MATERIAL_KEY = dm.MATERIAL_KEY
JOIN 
    NECCDMDEV.DIMENSIONS.DIM_SEASON_COLLECTION AS ds
    ON po.SEASON_COLLECTION_KEY = ds.SEASON_COLLECTION_KEY;
''', 
conn)

/tmp/ipykernel_4656/597121876.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  snowflake_query_v4 = pd.read_sql(


MK,POA
0, 1, 5950, Stock
0, 2, 5950, Stock
1, 3, 5950, Stock
2, 4, 5950, Stock

In [8]:
len(snowflake_query_base)

7822278

In [9]:
snowflake_query_base.to_csv('/home/azureuser/cloudfiles/code/Users/vlavanga/data/raw/snowflake_query_base.csv',index=False)

# Data Processing and CLeaning v4

In [5]:
snowflake_query_base = pd.read_csv('/home/azureuser/cloudfiles/code/Users/vlavanga/data/raw/snowflake_query_v5.2.csv')

In [6]:
snowflake_query_base.head()

,PO_MATERIAL_KEY,PURCHASE_ORDER_DOCUMENT,PO_CREATED_DATE,SALES_ORG,FABRIC_CONTENT_CODE_TEXT,TEAM,SILHOUETTE,SPORT,DIVISION_NAME,EAN_UPC,SEASON_CONSOLIDATION,ORDERED_QUANTITY
0,2d56d27d39a07fe26ea123a104d683150f2f17cf6ef856...,4500361594,20220225,2000,100% Polyester Knit,BROOKLYN NETS,T-SHIRT - SHORT SLEEVE,BASKETBALL,Apparel,196499534314,Program,6.0
1,2f5bafde67cef084dfe98f65542ad042a3931bf0f499dd...,4500510765,20251007,1700,Manmade fibers - Woven,NEW YORK YANKEES,5950 AFRAME,BASEBALL,Headwear,199420201507,Program,216.0
2,2253ef539c3fd88fcf8e0f35005dfc8890654cb1a2f5d6...,4500356699,20211221,1000,100% wool woven,ATLANTA BRAVES,5950,BASEBALL,Headwear,195501604779,Custom,2984.0
3,09487d883519be50d6a9b4197c0378ff41f538dc99e159...,4500168656,20160823,1000,100% wool woven,NEW YORK YANKEES,5950,BASEBALL,Headwear,886611303347,Custom,234.0
4,9f7d5315c570a2857f94fa68ae78019079206b6cd47fe8...,4500054321,20120221,1400,Polyester / Wool,PHILADELPHIA PHILLIES,5950,BASEBALL,Headwear,,Custom,149.0


In [8]:
print(snowflake_query_base.shape)
# total sum of ORDERED_QUANTITY (handles non-numeric values)
total_order_qty = pd.to_numeric(snowflake_query_base['ORDERED_QUANTITY'], errors='coerce').sum()
total_order_qty

(7796850, 14)


1622352842.0

In [9]:
print(len(snowflake_query_base))
snowflake_query_base_dedup = snowflake_query_base.drop_duplicates()
print(len(snowflake_query_base_dedup))

7796850
3375927


In [10]:
# total sum of ORDERED_QUANTITY (handles non-numeric values)
total_order_qty_dedup = pd.to_numeric(snowflake_query_base_dedup['ORDERED_QUANTITY'], errors='coerce').sum()
total_order_qty_dedup 

572831156.0

## [Silhouette[5950,970] -> Regions [United States] -> Fabric Type [Woven] -> Division[Headwear] -> Season_Consolidation[Stock,Custom,Program]] -> [Sport,Team]

## 5950 -> United States -> Woven -> Headwear -> Stock

In [7]:
print(len(snowflake_query_base))
snowflake_query_base = snowflake_query_base[~snowflake_query_base['PURCHASE_ORDER_DOCUMENT'].astype(str).str.startswith('6')]
print(len(snowflake_query_base))

7796157
7796149


In [8]:
# Drop Silhouette column, EAN_UPC, Market Segment, Product Hierarchy, Material Group,
snowflake_query_base = snowflake_query_base.drop(columns=['PO_MATERIAL_KEY','PURCHASE_ORDER_DOCUMENT', 'EAN_UPC'])


In [13]:
snowflake_query_base['SALES_ORG'].unique()

array(['2000', '1000', '5000', '1200', '3200', '3100', '3000', '1700',
       '3500', '4000', '1400', '1500', '3300', '4100', '1300', '2100',
       '4147', '3247', '2047', '4047', '2200'], dtype=object)

In [9]:
#snowflake_query_base['SALES_ORG'] = snowflake_query_base['SALES_ORG'].fillna(0)
#snowflake_query_base['SALES_ORG'] = snowflake_query_base['SALES_ORG'].astype(int)   

snowflake_query_base['ORDERED_QUANTITY'] = snowflake_query_base['ORDERED_QUANTITY'].astype(int)

In [10]:
fabric_text = snowflake_query_base['FABRIC_CONTENT_CODE_TEXT'].unique()

wovens = []
knits = []
poly = []
other = []
error_values = []

for word in fabric_text:
    try:
        val = str(word).lower()
        if "woven" in val:
            wovens.append(word)
        elif "knit" in val:
            knits.append(word)
        elif "polyester" in val or "poly" in val:
            poly.append(word)
        else:
            other.append(word)
    except:
        error_values.append(word)

In [11]:
def classify_fabric(value):
    try:
        val = str(value).lower()
        if value in wovens:
            return "Wovens"
        elif value in knits:
            return "Knits"
        elif value in val or "poly" in poly:
            return "Polyester"
        else:
            return "Other"
    except:
        return "Other"


insert_index = snowflake_query_base.columns.get_loc('FABRIC_CONTENT_CODE_TEXT') + 1
snowflake_query_base.insert(insert_index, 'FABRIC_TYPE', snowflake_query_base['FABRIC_CONTENT_CODE_TEXT'].apply(classify_fabric))

In [12]:
snowflake_query_base.head()

,PO_CREATED_DATE,SALES_ORG,FABRIC_CONTENT_CODE_TEXT,FABRIC_TYPE,TEAM,SILHOUETTE,SPORT,DIVISION_NAME,SEASON_CONSOLIDATION,ORDERED_QUANTITY
0,20220225,2000,100% Polyester Knit,Knits,BROOKLYN NETS,T-SHIRT - SHORT SLEEVE,BASKETBALL,Apparel,Program,6
1,20251007,1700,Manmade fibers - Woven,Wovens,NEW YORK YANKEES,5950 AFRAME,BASEBALL,Headwear,Program,216
2,20211221,1000,100% wool woven,Wovens,ATLANTA BRAVES,5950,BASEBALL,Headwear,Custom,2984
3,20160823,1000,100% wool woven,Wovens,NEW YORK YANKEES,5950,BASEBALL,Headwear,Custom,234
4,20120221,1400,Polyester / Wool,Other,PHILADELPHIA PHILLIES,5950,BASEBALL,Headwear,Custom,149


In [13]:
df_sales_org_new = pd.read_csv('/home/azureuser/cloudfiles/code/Users/vlavanga/data/raw/sales_org_info.csv')
df_sales_org_old = pd.read_csv('/home/azureuser/cloudfiles/code/Users/vlavanga/data/raw/sales_org_mapping.csv')

In [14]:
def classify_sales_org_name(value):
    try:
        if value in df_sales_org_new['SALES_ORGANIZATION'].values:
            return df_sales_org_new[df_sales_org_new['SALES_ORGANIZATION'] == value]['ENGLISH_NAME'].values[0]
        else:
            return "Other"
    except:
        return "Other"
    
def classify_sales_country(value):
    try:
        if value in df_sales_org_new['SALES_ORGANIZATION'].values:
            return df_sales_org_new[df_sales_org_new['SALES_ORGANIZATION'] == value]['COUNTRY'].values[0]
        else:
            return "Other"
    except:
        return "Other"
    
def classify_sales_region(value):
    try:
        if value in df_sales_org_new['SALES_ORGANIZATION'].values:
            return df_sales_org_new[df_sales_org_new['SALES_ORGANIZATION'] == value]['REGION'].values[0]
        else:
            return "Other"
    except:
        return "Other"
    


In [15]:
insert_index = snowflake_query_base.columns.get_loc('SALES_ORG') + 1
snowflake_query_base.insert(insert_index, 'SALES_ORG_NAME', snowflake_query_base['SALES_ORG'].apply(classify_sales_org_name))
print('org name complete')

insert_index = snowflake_query_base.columns.get_loc('SALES_ORG') + 1
snowflake_query_base.insert(insert_index, 'COUNTRY', snowflake_query_base['SALES_ORG'].apply(classify_sales_country))
print('country complete')


insert_index = snowflake_query_base.columns.get_loc('SALES_ORG') + 1
snowflake_query_base.insert(insert_index, 'REGION', snowflake_query_base['SALES_ORG'].apply(classify_sales_region))
print('country region')

org name complete
country complete
country region


In [16]:
snowflake_query_base.head()

,PO_CREATED_DATE,SALES_ORG,REGION,COUNTRY,SALES_ORG_NAME,FABRIC_CONTENT_CODE_TEXT,FABRIC_TYPE,TEAM,SILHOUETTE,SPORT,DIVISION_NAME,SEASON_CONSOLIDATION,ORDERED_QUANTITY
0,20220225,2000,EMEA,GB : United Kingdom,EMEA,100% Polyester Knit,Knits,BROOKLYN NETS,T-SHIRT - SHORT SLEEVE,BASKETBALL,Apparel,Program,6
1,20251007,1700,Emerging Markets,US : USA,Latin America,Manmade fibers - Woven,Wovens,NEW YORK YANKEES,5950 AFRAME,BASEBALL,Headwear,Program,216
2,20211221,1000,North America,US : USA,United States,100% wool woven,Wovens,ATLANTA BRAVES,5950,BASEBALL,Headwear,Custom,2984
3,20160823,1000,North America,US : USA,United States,100% wool woven,Wovens,NEW YORK YANKEES,5950,BASEBALL,Headwear,Custom,234
4,20120221,1400,Emerging Markets,US : USA,Latin America,Polyester / Wool,Other,PHILADELPHIA PHILLIES,5950,BASEBALL,Headwear,Custom,149


In [17]:
snowflake_query_base.to_csv('/home/azureuser/cloudfiles/code/Users/vlavanga/data/processed/snowflake_v5.2_pre_ts.csv', index=False)

In [6]:
df = pd.read_csv('/home/azureuser/cloudfiles/code/Users/vlavanga/data/processed/snowflake_v4_pre_ts.csv')

## [Silhouette[5950,970] -> Regions [United States] -> Fabric Type [Woven] -> Division[Headwear] -> Season_Consolidation[Stock,Custom,Program]] -> [Sport,Team]

## 5950 -> United States -> Woven -> Headwear -> Stock

## Big picture categories
Region[North America] -> Sales_Org[United States, US Retail, Canada, 5th & Ocean] -> Silhouette[5950, Low Profile 5950, 970, 950, 3930, 940, 920]

## 2nd Category
Region[North America] -> Sales_Org[United States, US Retail, Canada, 5th & Ocean, Mexico] -> Silhouette[5950, Low Profile 5950, 970, 950, 3930, 940, 920] -> Season Consolidation[Stock, Program, Custom]

# Upload and Store Data

In [13]:
snowflake_query_v4 = pd.read_csv("/home/azureuser/cloudfiles/code/Users/vlavanga/data/processed/snowflake_v4_pre_ts.csv")

# Data Processing v5.1

In [23]:
df = pd.read_csv('/home/azureuser/cloudfiles/code/Users/vlavanga/data/processed/snowflake_v5_pre_ts.csv')
df = df[~df['SALES_ORG_NAME'].isin(['47 Japan','47 New Zealand','47 UK','47 Australia'])]
df.head()

KeyboardInterrupt: 

In [ ]:
def edit_silhouette(value):    
    silhouette = ['5950','920','940','3930','950']
    if value in silhouette:
        return value
    else:
        return "Other"


insert_index = df.columns.get_loc('SILHOUETTE') + 1
df.insert(insert_index, 'SILHOUETTE_UPDATED', df['SILHOUETTE'].apply(edit_silhouette))

In [ ]:
df['SILHOUETTE_UPDATE'].unique()

array(['Other', '940', '5950', '3930', '950', '920'], dtype=object)

In [ ]:
df.head()

,PO_CREATED_DATE,SALES_ORG,REGION,COUNTRY,SALES_ORG_NAME,FABRIC_CONTENT_CODE_TEXT,FABRIC_TYPE,TEAM,SILHOUETTE,SILHOUETTE_UPDATE,SPORT,DIVISION_NAME,SEASON_CONSOLIDATION,ORDERED_QUANTITY
0,20231003,3200,Japan,JP : Japan,Japan,100% Cotton woven,Wovens,YOMIURI GIANTS,LP5950,Other,BASEBALL,Headwear,Custom,72
1,20231003,3200,Japan,JP : Japan,Japan,100% Cotton woven,Wovens,YOMIURI GIANTS,LP5950,Other,BASEBALL,Headwear,Custom,72
2,20231109,4100,Emerging Markets,NZ : New Zealand,New Zealand,Cotton - Woven,Wovens,LAS VEGAS RAIDERS,940,940,FOOTBALL,Headwear,Program,80
3,20231103,1000,North America,US : USA,United States,Manmade fibers - Woven,Wovens,RANCHO CUCAMONGA QUAKES,5950,5950,BASEBALL,Headwear,Program,1
4,20231103,1000,North America,US : USA,United States,Manmade fibers - Woven,Wovens,RENO ACES,5950,5950,BASEBALL,Headwear,Program,1


In [62]:
len(df)

7741722

# Data Processing v5.2

In [18]:
import pandas as pd
df = pd.read_csv('/home/azureuser/cloudfiles/code/Users/vlavanga/data/processed/snowflake_v5.2_pre_ts.csv')
df = df[~df['SALES_ORG_NAME'].isin(['47 Japan','47 New Zealand','47 UK','47 Australia'])]
print("With PO value with startswith 6",len(df))
df.head()

With PO value with startswith 6 7795061


,PO_CREATED_DATE,SALES_ORG,REGION,COUNTRY,SALES_ORG_NAME,FABRIC_CONTENT_CODE_TEXT,FABRIC_TYPE,TEAM,SILHOUETTE,SPORT,DIVISION_NAME,SEASON_CONSOLIDATION,ORDERED_QUANTITY
0,20220225,2000,EMEA,GB : United Kingdom,EMEA,100% Polyester Knit,Knits,BROOKLYN NETS,T-SHIRT - SHORT SLEEVE,BASKETBALL,Apparel,Program,6
1,20251007,1700,Emerging Markets,US : USA,Latin America,Manmade fibers - Woven,Wovens,NEW YORK YANKEES,5950 AFRAME,BASEBALL,Headwear,Program,216
2,20211221,1000,North America,US : USA,United States,100% wool woven,Wovens,ATLANTA BRAVES,5950,BASEBALL,Headwear,Custom,2984
3,20160823,1000,North America,US : USA,United States,100% wool woven,Wovens,NEW YORK YANKEES,5950,BASEBALL,Headwear,Custom,234
4,20120221,1400,Emerging Markets,US : USA,Latin America,Polyester / Wool,Other,PHILADELPHIA PHILLIES,5950,BASEBALL,Headwear,Custom,149


In [19]:
def edit_silhouette(value):    
    silhouette = ['5950','920','940','3930','950']
    if value in silhouette:
        return value
    if value == 'NONE':
        return 'Unknown'
    else:
        return "Other"


insert_index = df.columns.get_loc('SILHOUETTE') + 1
df.insert(insert_index, 'SILHOUETTE_UPDATED', df['SILHOUETTE'].apply(edit_silhouette))

In [20]:
sports_info = {
    'COBRANDED BASEBALL': 'BASEBALL',
    'COBRANDED BASKETBALL': 'BASKETBALL',
    'COBRANDED RACING': 'RACING',
    'COBRANDED CORE': 'CORE',
    'COBRANDED NON LICENSED': 'NON LICENSED',
    'COBRANDED COLLEGE': 'COLLEGE',
    'COBRANDED FOOTBALL': 'FOOTBALL',
    'COBRANDED HOCKEY': 'HOCKEY',
    'COBRANDED SOCCER': 'SOCCER'
}

In [21]:
def edit_sport(value):
    try:
        if value in sports_info.keys():
            return sports_info[value]
        else:
            return value
    except:
        return value

In [22]:
insert_index = df.columns.get_loc('SPORT') + 1
df.insert(insert_index, 'SPORT_UPDATED', df['SPORT'].apply(edit_sport))
df.head()

,PO_CREATED_DATE,SALES_ORG,REGION,COUNTRY,SALES_ORG_NAME,FABRIC_CONTENT_CODE_TEXT,FABRIC_TYPE,TEAM,SILHOUETTE,SILHOUETTE_UPDATED,SPORT,SPORT_UPDATED,DIVISION_NAME,SEASON_CONSOLIDATION,ORDERED_QUANTITY
0,20220225,2000,EMEA,GB : United Kingdom,EMEA,100% Polyester Knit,Knits,BROOKLYN NETS,T-SHIRT - SHORT SLEEVE,Other,BASKETBALL,BASKETBALL,Apparel,Program,6
1,20251007,1700,Emerging Markets,US : USA,Latin America,Manmade fibers - Woven,Wovens,NEW YORK YANKEES,5950 AFRAME,Other,BASEBALL,BASEBALL,Headwear,Program,216
2,20211221,1000,North America,US : USA,United States,100% wool woven,Wovens,ATLANTA BRAVES,5950,5950,BASEBALL,BASEBALL,Headwear,Custom,2984
3,20160823,1000,North America,US : USA,United States,100% wool woven,Wovens,NEW YORK YANKEES,5950,5950,BASEBALL,BASEBALL,Headwear,Custom,234
4,20120221,1400,Emerging Markets,US : USA,Latin America,Polyester / Wool,Other,PHILADELPHIA PHILLIES,5950,5950,BASEBALL,BASEBALL,Headwear,Custom,149


In [23]:
df['SPORT_UPDATED'].unique()

array(['BASKETBALL', 'BASEBALL', 'FOOTBALL', 'RUGBY', 'NONE', 'COLLEGE',
       'SOCCER', 'RACING', 'GOLF', 'ENTERTAINMENT', 'HOCKEY', 'WRESTLING',
       'CORE', 'NON LICENSED', 'LACROSSE', 'CRICKET', 'TENNIS', 'BOXING',
       nan, 'ANIMATED CHARACTER', 'E SPORT', 'CYCLING', 'WATER SPORTS',
       'SNOWSPORTS', 'GAELIC FOOTBALL', 'VOLLEYBALL', 'SOFTBALL',
       'HANDBALL'], dtype=object)

In [25]:
len(df)

7795061

In [24]:
df = df.drop(columns=['SALES_ORG','FABRIC_CONTENT_CODE_TEXT','SILHOUETTE','SPORT'])

In [33]:
df.head()

,PO_CREATED_DATE,REGION,COUNTRY,SALES_ORG_NAME,FABRIC_TYPE,TEAM,SILHOUETTE_UPDATED,SPORT_UPDATED,DIVISION_NAME,MARKET_SEGMENT,GENDER,SEASON_CONSOLIDATION,ORDERED_QUANTITY
0,20230412,EMEA,GB : United Kingdom,EMEA,Other,HOUSTON ASTROS,5950,BASEBALL,Headwear,LIFESTYLE,MALE,Custom,72
1,20200206,North America,US : USA,United States,Knits,ARIZONA DIAMONDBACKS,5950,BASEBALL,Headwear,ON FIELD,MALE,Program,288
2,20190215,North America,US : USA,United States,Knits,MICHIGAN WOLVERINES,Other,COLLEGE,Headwear,FANGEAR,BOYS,Program,48
3,20150206,North America,US : USA,5th & Ocean,Wovens,NEW YORK JETS,Other,FOOTBALL,Apparel,WOMENS APPAREL - UPSTAIRS,YOUTH,Custom,53
4,20210303,North America,US : USA,United States,Wovens,NEW YORK YANKEES,5950,BASEBALL,Headwear,LIFESTYLE,MALE,Custom,64


In [34]:
len(df)

7795754

In [26]:
df.to_csv('/home/azureuser/cloudfiles/code/Users/vlavanga/data/processed/snowflake_v5.2_pre_ts.csv', index=False)